IMPORTAÇÃO

In [ ]:
from config_path import outros
import pandas as pd

CARGA

In [74]:
col_corte = ['DATA', 'COD', 'DESCRIÇÃO','RUA', 'PREDIO', 'APTO', 'QTDE CORTE', 'VL CORTE','MOTIVO','FUNCIONARIO']
df_corte_geral = pd.read_excel(outros.ou_corte, sheet_name= "CORTE 2025", usecols= col_corte) 

col_estoque = ['Código', 'Estoque', 'Qtde Pedida','Bloqueado(Qt.Bloq.-Qt.Avaria)', 'Qt.Avaria']
df_estoque = pd.read_excel(outros.ou_86, usecols= col_estoque)

TRATAMENTO

In [75]:
filtro_data = df_corte_geral['DATA'].max() - pd.Timedelta(days= 20)
df_filtrado = df_corte_geral.loc[df_corte_geral['DATA'] > filtro_data].copy()

grupo = df_filtrado.groupby(["COD", "DESCRIÇÃO"]).agg(
    DIAS_CORTE = ("DATA", "nunique"),
    TOTAL_CORTE = ("QTDE CORTE", "sum"),
    VL_CORTE = ("VL CORTE", lambda x: round(x.sum(), 2))
).reset_index().astype({"COD": "int", "TOTAL_CORTE": "int"}).sort_values('DIAS_CORTE', ascending=False)

df_final = grupo.merge(df_estoque, left_on='COD', right_on= 'Código', how= 'left').drop(columns= 'Código')
df_final = df_final.fillna({'Estoque':0, "Qtde Pedida":0}).astype({"Estoque": "int", "Qtde Pedida": "int"})
df_final["BLOQUEIO"] = df_final["Bloqueado(Qt.Bloq.-Qt.Avaria)"].fillna(0).astype(int) + df_final["Qt.Avaria"].fillna(0).astype(int)
df_final = df_final.drop(columns= ['Bloqueado(Qt.Bloq.-Qt.Avaria)', 'Qt.Avaria'])
df_final ['DIFERENCA'] = df_final["Estoque"] - df_final["BLOQUEIO"] 


CONSULTA

In [78]:
caso_1 = df_final.loc[(df_final['DIFERENCA'] > 0) & (df_final['DIAS_CORTE'] > 2)]

caso_2 = df_filtrado.loc[(df_filtrado['COD'] == 152001)]

display(caso_1.head(8))
display(caso_2)


,COD,DESCRIÇÃO,DIAS_CORTE,TOTAL_CORTE,VL_CORTE,Estoque,Qtde Pedida,BLOQUEIO,DIFERENCA
0,152001,SAL CISNE CHURRASCO 1KG,6,8,1102.42,2,0,1,1
6,459577,RESF QUEIJO MUSSARELA ITALAC,3,0,7.07,162,642,80,82


,DATA,COD,DESCRIÇÃO,RUA,PREDIO,APTO,QTDE CORTE,VL CORTE,MOTIVO,FUNCIONARIO
15908,2025-11-05,152001,SAL CISNE CHURRASCO 1KG,4.0,70.0,101.0,1.0,130.89,QUANTIDADE INFERIOR DA OS,YAGO RODRIGUES DA SILVA
15963,2025-11-06,152001,SAL CISNE CHURRASCO 1KG,4.0,70.0,101.0,1.0,134.74,QUANTIDADE INFERIOR DA OS,YAGO RODRIGUES DA SILVA
15985,2025-11-07,152001,SAL CISNE CHURRASCO 1KG,4.0,70.0,101.0,1.0,140.35,QUANTIDADE INFERIOR DA OS,TAINA MEDEIROS RIBEIRO
15986,2025-11-07,152001,SAL CISNE CHURRASCO 1KG,4.0,70.0,101.0,1.0,134.53,QUANTIDADE INFERIOR DA OS,YAGO RODRIGUES DA SILVA
16065,2025-11-11,152001,SAL CISNE CHURRASCO 1KG,60.0,1.0,508.0,1.0,147.37,QUANTIDADE INFERIOR DA OS,CARLOS ALBERTO SANTOS DA SILVA FILHO
16123,2025-11-12,152001,SAL CISNE CHURRASCO 1KG,60.0,1.0,508.0,1.0,138.00,QUANTIDADE INFERIOR DA OS,YAGO RODRIGUES DA SILVA
16131,2025-11-12,152001,SAL CISNE CHURRASCO 1KG,60.0,1.0,508.0,1.0,138.27,QUANTIDADE INFERIOR DA OS,TAINA MEDEIROS RIBEIRO
16302,2025-11-19,152001,SAL CISNE CHURRASCO 1KG,60.0,1.0,508.0,1.0,138.27,QUANTIDADE INFERIOR DA OS,TAINA MEDEIROS RIBEIRO
